 # Feature Store Tour - Scala API
 
This notebook contains a tour/reference for the feature store Scala API on hopsworks. We will go over best practices for using the API as well as common pitfalls.
 
The notebook is designed to be used in combination with the Feature Store Tour on Hopsworks, it assumes that you have run the following feature engineering job: [job](https://github.com/logicalclocks/hops-examples/tree/master/featurestore) (**the job is added automatically when you start the feature store tour in Hopsworks. You can run the job by going to the 'Jobs' tab to the left in the Hopsworks project home page**). 

Which will produce the following model of feature groups in your project's feature store:

![Feature Store Model](./images/model.png "Feature Store Model")

In this notebook we will run queries over this feature store model. We will also create new feature groups and training datasets.

We will go from (1) features to (2) training datasets to (3) A trained model

## Feature Store 101

The simplest way to think about the feature store is as a central place to store curated /features/ within an organization. A feature is a measurable property of some phenomenon. It could be for example an image-pixel, a word from a piece of text, the age of a person, a coordinate emitted from a sensor, or an aggregate value like the average number of purchases within the last hour.

A feature store is a data management layer for machine learning that can optimize the machine learning workflow and provide an interface between data engineering and data science.

![Feature Store Overview](./images/overview.png "Feature Store Overview")

A feature store is not a pure service concept, it goes hand-in-hand with feature computation. Feature engineering is the process of transforming raw data into a format that is compatible and understandable for predictive models.

There are two interfaces to the feature store:

- Writing to the feature store, at the end of the feature engineering pipeline the features are written to the feature store, e.g:

```scala
val rawData = spark.read.format("csv").load(filename)
val polynomialFeatures = rawData.map((x: Float) => scala.math.pow(x, 2))
import io.hops.util.Hops
Hops.insertIntoFeaturegroup(
    polynomialFeatures, 
    spark, 
    "polynomial_features",
    featurestore,
    featuregroupVersion,
    mode,
    descriptiveStats, 
    featureCorr,
    featureHistograms, 
    clusterAnalysis, 
    statColumns, 
    numBins,
    corrMethod, 
    numClusters
)
```
- Reading from the feature store, to train a model on a set of features, the features can be read from the feature store, e.g:

```scala
import io.hops.util.Hops
val features = List("team_budget", "average_attendance", "average_player_age")
val featuresDf = Hops.getFeatures(spark, features, Hops.getProjectFeaturestore)
```

As a data engineer/data scientist, you can think of the feature store as a middle-layer. Once you have computed a set of features, instead of writing them locally to a csv file, insert them in the feature store so that the features can get documented/versioned, backfilled, **and so that your colleagues can re-use your features!** 

## Imports

The hops library is automatically installed in all Hopsworks-projects.You can find API documentation [here](http://snurran.sics.se/hops/hops-util-javadoc/0.6.0-SNAPSHOT/).

In [1]:
import io.hops.util.Hops
import scala.collection.JavaConversions._
import collection.JavaConverters._

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
31,application_1549128638243_0036,spark,idle,Link,Link,✔


SparkSession available as 'spark'.
import io.hops.util.Hops
import scala.collection.JavaConversions._
import collection.JavaConverters._


## Get The Name of The Project's Feature Store

Each project with the feature store service enabled automatically gets its own feature store created. This feature store is only accessible within the project unless you decide to share it with other projects. The name of the feature store is `<project_name>_featurestore`, and you can get the name with the API method `getProjectFeaturestore`. 

In [2]:
Hops.getProjectFeaturestore

res1: String = demo_featurestore_admin000_featurestore


## Get a List of All Feature Stores Accessible in the Current Project 

Feature Stores can be shared across projects in a multi-tenant manner, just like any Hopsworks-dataset can. You can read more about sharing datasets at [hops.io](hops.io), but in essence to share a dataset you just have to right click on it in your project. The features and featuregroups in the feature store are stored in a dataset called `<project_name>_featurestore.db` in your project.

![Share Feature Store](./images/share_featurestore.png "Share Feature Store")

The training datasets in the feature store are stored in a dataset called `project_name_Training_Datasets`. 

![Share Feature Store](./images/share_featurestore.png "Share Feature Store")

Typically, if you are sharing a feature store with another project, you want to share both the `<project_name>_featurestore.db` dataset and the `project_name_Training_Datasets` dataset.

To list all feature stores accessible in the current project, you can use the API method `getProjectFeaturestores()`. You can also view the list of accessible feature stores in the feature registry UI:

![Share Feature Store](./images/select_fs.png "Share Feature Store")

By using multiple feature stores and feature store sharing across projects you can enforce access rights to features.

![Multi-Tenant Feature Stores](./images/multitenant.png "Multi-Tenant Feature Stores")

In [3]:
Hops.getProjectFeaturestores

res2: java.util.List[String] = [demo_featurestore_admin000_featurestore]


## Querying The Feature Store

The feature store can be queried programmatically and with raw SQL. When you query the feature store programmatically, the library will infer how to fetch the different features using a **query planner**. 

![Feature Store Query Planner](./images/query_optimizer.png "Feature Store Query Planner")

When interacting with the feature store it is sufficient to be familiar with three concepts:

- The **feature**, this refer to an individual versioned and documented feature in the feature store, e.g the age of a person.
- The **feature group**, this refer to a documented and versioned group of features stored as a Hive table that is linked to a specific Spark/Numpy/Pandas job that takes in raw data and outputs the computed features.
- The **training dataset**, this refer to a versioned and managed dataset of features, stored in HopsFS as tfrecords, .csv, .tsv, or parquet.

A feature group contains a group of features and a training dataset contains a set of features, potentially from many different feature groups.

![Feature Store Concepts](./images/concepts.png "Feature Store Contents")

When you query the feature store you will always get back the results in a spark dataframe. This is for scalability reasons. If the dataset is small and you want to work with it in memory you can convert it into a pandas dataframe or a numpy matrix using one line of code as we will demonstrate later on in this notebook.

### Fetch an Individual Feature

When retrieving a single feature from the featurestore, the hops-util-py library will infer in which feature group the feature belongs to by querying the metastore, but you can also explicitly specify which featuregroup and version to query. 

If there are multiple features of the same name in the featurestore, it is required to specify enough information to uniquely identify the feature (e.g specify feature group and version). If no featurestore is provided it will default to the project's featurestore.

To read an individual feature, use the method `getFeature`

In [4]:
Hops.getFeature(spark, "team_budget", Hops.getProjectFeaturestore).show(5)

+-----------+
|team_budget|
+-----------+
|  12957.076|
|  2403.3704|
|  3390.3755|
|  13547.429|
|   9678.333|
+-----------+
only showing top 5 rows



You can also explicitly specify the feature store, feature group and version:

In [5]:
Hops.getFeature(spark, "team_budget", Hops.getProjectFeaturestore, "teams_features", 1).show(5)

+-----------+
|team_budget|
+-----------+
|  12957.076|
|  2403.3704|
|  3390.3755|
|  13547.429|
|   9678.333|
+-----------+
only showing top 5 rows



### Fetch an Entire Feature Group

You can get an entire featuregroup from the API. If no feature store is provided the API will default to the project's feature store, if no version is provided it will default to version 1 of the feature group.

In [6]:
Hops.getFeaturegroup(spark, "teams_features", Hops.getProjectFeaturestore, 1).show(5)

+-----------+-------+-------------+
|team_budget|team_id|team_position|
+-----------+-------+-------------+
|  12957.076|      1|            1|
|  2403.3704|      2|            2|
|  3390.3755|      3|            3|
|  13547.429|      4|            4|
|   9678.333|      5|            5|
+-----------+-------+-------------+
only showing top 5 rows



### Fetch A Set of Features

When retrieving a list of features from the featurestore, the hops-util-py library will infer which featuregroup the features belongs to by querying the metastore. If the features reside in different featuregroups, the library will also try to infer how to join the features together based on common columns. If the JOIN query cannot be inferred due to existence of multiple features with the same name or non-obvious JOIN query, the user need to supply enough information to the API call to be able to query the featurestore. If the user already knows the JOIN query it can also run featurestore.sql(joinQuery) directly (an example of this is shown further down in this notebook). If no featurestore is provided the API will default to the project's featurestore.

Example of querying the feature store for a list of features without specifying the feature groups and feature store:

In [7]:
val features = List("team_budget", "average_attendance", "average_player_age")
Hops.getFeatures(spark, features, Hops.getProjectFeaturestore).show(5)

features: List[String] = List(team_budget, average_attendance, average_player_age)
+-----------+------------------+------------------+
|team_budget|average_attendance|average_player_age|
+-----------+------------------+------------------+
|    7307.94|         19595.973|             24.85|
|   7326.092|          6462.462|             25.45|
|   3555.235|          7226.672|              25.4|
|  910.39325|         3189.8455|             25.91|
|  12474.419|          9405.213|             25.78|
+-----------+------------------+------------------+
only showing top 5 rows



We can also explicitly specify the feature groups where the features reside. Either the feature groups and versions can be specified by prepending feature names with `<feature group name>_<feature group version.`, or by providing a Map with entries of `<feature group name> -> <feature group version>`:

In [8]:
val features = List("teams_features_1.team_budget", 
                    "attendances_features_1.average_attendance", 
                    "players_features_1.average_player_age")
Hops.getFeatures(spark, features, Hops.getProjectFeaturestore).show(5)

features: List[String] = List(teams_features_1.team_budget, attendances_features_1.average_attendance, players_features_1.average_player_age)
+-----------+------------------+------------------+
|team_budget|average_attendance|average_player_age|
+-----------+------------------+------------------+
|    7307.94|         19595.973|             24.85|
|   7326.092|          6462.462|             25.45|
|   3555.235|          7226.672|              25.4|
|  910.39325|         3189.8455|             25.91|
|  12474.419|          9405.213|             25.78|
+-----------+------------------+------------------+
only showing top 5 rows



In [9]:
val featuregroupsMap = Map[String, Integer](
    "teams_features"->1,
    "attendances_features"->1,
    "players_features"->1
)
val javaFeaturegroupsMap = new java.util.HashMap[String, Integer](featuregroupsMap)
Hops.getFeatures(spark, features, Hops.getProjectFeaturestore, javaFeaturegroupsMap).show(5)

featuregroupsMap: scala.collection.immutable.Map[String,Integer] = Map(teams_features -> 1, attendances_features -> 1, players_features -> 1)
javaFeaturegroupsMap: java.util.HashMap[String,Integer] = {attendances_features=1, players_features=1, teams_features=1}
+-----------+------------------+------------------+
|team_budget|average_attendance|average_player_age|
+-----------+------------------+------------------+
|    7307.94|         19595.973|             24.85|
|   7326.092|          6462.462|             25.45|
|   3555.235|          7226.672|              25.4|
|  910.39325|         3189.8455|             25.91|
|  12474.419|          9405.213|             25.78|
+-----------+------------------+------------------+
only showing top 5 rows



If you have a lot of name collisions and it is not obvious how to infer the JOIN query to get the features from the feature store. You can explicitly specify the argument `joinKey` to the API (or you can provide the entire SQL query using the API method `.sql` as we will demonstrate later on in the notebook)

In [10]:
Hops.getFeatures(spark, features, Hops.getProjectFeaturestore, javaFeaturegroupsMap, "team_id").show(5)

+-----------+------------------+------------------+
|team_budget|average_attendance|average_player_age|
+-----------+------------------+------------------+
|    7307.94|         19595.973|             24.85|
|   7326.092|          6462.462|             25.45|
|   3555.235|          7226.672|              25.4|
|  910.39325|         3189.8455|             25.91|
|  12474.419|          9405.213|             25.78|
+-----------+------------------+------------------+
only showing top 5 rows



#### Advanced Eamples of Fetching Sets of Features and Common Pitfalls

Getting 12 features from 4 different feature groups:

In [11]:
val features1 = List("team_budget", "average_attendance", "average_player_age", "team_position", 
                     "sum_attendance", "average_player_rating", "average_player_worth", "sum_player_age", 
                     "sum_player_rating", "sum_player_worth", "sum_position", "average_position")
Hops.getFeatures(spark, features1, Hops.getProjectFeaturestore).show(5)

features1: List[String] = List(team_budget, average_attendance, average_player_age, team_position, sum_attendance, average_player_rating, average_player_worth, sum_player_age, sum_player_rating, sum_player_worth, sum_position, average_position)
+-----------+------------------+------------------+-------------+--------------+---------------------+--------------------+--------------+-----------------+----------------+------------+----------------+
|team_budget|average_attendance|average_player_age|team_position|sum_attendance|average_player_rating|average_player_worth|sum_player_age|sum_player_rating|sum_player_worth|sum_position|average_position|
+-----------+------------------+------------------+-------------+--------------+---------------------+--------------------+--------------+-----------------+----------------+------------+----------------+
|    7307.94|         19595.973|             24.85|            6|     391919.47|            1311.2384|           1435.2465|        2485.0|     

##### Example Errors

Lets look at an example of a common error that can occur when you query the feature store.

If you try to query the feature store for a feature that exists in multiple feature groups, it is impossible for the query planner to infer from which feature group to fetch the feature so it will throw an exception. When this error happen you should specify which feature group to fetch from so that the query planner knows how to get the feature.

**Note**: <font color='red'>This cell should fail, don't panic :)</font>

In [12]:
val features1 = List("team_budget", "team_id")
Hops.getFeatures(spark, features1, Hops.getProjectFeaturestore).show(5)

java.lang.IllegalArgumentException: Found the feature with name: team_id in more than one of the featuregroups of the featurestore demo_featurestore_admin000_featurestore please specify featuregroup that you want to get the feature from. The matched featuregroups are: season_scores_features_1, teams_features_spanish_1, teams_features_1, numpy_test_example_1, python_test_example_1, pandas_test_example_1, players_features_1, games_features_1, attendances_features_1, teams_features_spanish_2
  at io.hops.util.featurestore.FeaturestoreHelper.findFeature(FeaturestoreHelper.java:413)
  at io.hops.util.featurestore.FeaturestoreHelper.findFeaturegroupsThatContainsFeatures(FeaturestoreHelper.java:273)
  at io.hops.util.Hops.doGetFeatures(Hops.java:1807)
  at io.hops.util.Hops.getFeatures(Hops.java:1789)
  ... 54 elided



Let's fix the error: 

In [13]:
val features1 = List("team_budget", "team_id")
val featuregroupsMap = Map[String, Integer](
    "teams_features"->1
)
val javaFeaturegroupsMap = new java.util.HashMap[String, Integer](featuregroupsMap)
Hops.getFeatures(spark, features1, Hops.getProjectFeaturestore, javaFeaturegroupsMap).show(5)

features1: List[String] = List(team_budget, team_id)
featuregroupsMap: scala.collection.immutable.Map[String,Integer] = Map(teams_features -> 1)
javaFeaturegroupsMap: java.util.HashMap[String,Integer] = {teams_features=1}
+-----------+-------+
|team_budget|team_id|
+-----------+-------+
|  12957.076|      1|
|  2403.3704|      2|
|  3390.3755|      3|
|  13547.429|      4|
|   9678.333|      5|
+-----------+-------+
only showing top 5 rows



Another common error is that you try to fetch features from feature groups that are not compatible, they do not got any natural join column. Typically in this case you need to either provide the join key your self or use SQL directly with `featurestore.sql()`.


**Note**: <font color='red'>This cell should fail, don't panic :)</font>

In [14]:
val features1 = List("team_budget", "score")
Hops.getFeatures(spark, features1, Hops.getProjectFeaturestore).show(5)

java.lang.IllegalArgumentException: Could not find any common columns in featuregroups to join on, searched through the following featuregroups: teams_features, games_features
  at io.hops.util.featurestore.FeaturestoreHelper.getJoinColumn(FeaturestoreHelper.java:555)
  at io.hops.util.Hops.doGetFeatures(Hops.java:1808)
  at io.hops.util.Hops.getFeatures(Hops.java:1789)
  ... 54 elided



Lets fix the error:

In [15]:
Hops.queryFeaturestore(spark,
    "SELECT team_budget, score " +
    "FROM teams_features_1 JOIN games_features_1 ON " +
    "games_features_1.home_team_id = teams_features_1.team_id", Hops.getProjectFeaturestore).show()

+-----------+-----+
|team_budget|score|
+-----------+-----+
|  11296.577|    1|
|   4969.735|    3|
|  21319.533|    2|
|  15072.062|    1|
|  12957.076|    3|
|   760.8729|    1|
|  20347.281|    2|
|   2248.776|    3|
|  11296.577|    1|
|  12514.562|    1|
|  16758.066|    1|
|  13547.429|    3|
|  11169.979|    1|
|  1583.5911|    1|
|   3555.235|    1|
|  8154.7256|    1|
|   2248.776|    1|
|  7683.7227|    1|
|  910.39325|    1|
|   9775.455|    1|
+-----------+-----+
only showing top 20 rows



### Free Text SQL Query from the Feature Store

For complex queries that cannot be inferred by the helper functions, enter the sql directly to the method `featurestore.sql()` it will default to the project specific feature store but you can also specify it explicitly. If you are proficient in SQL, this is the most efficient and preferred way to query the feature store.

In [16]:
Hops.queryFeaturestore(spark,
                       "SELECT * FROM teams_features_1 WHERE team_position < 5", 
                       Hops.getProjectFeaturestore).show()

+-----------+-------+-------------+
|team_budget|team_id|team_position|
+-----------+-------+-------------+
|  12957.076|      1|            1|
|  2403.3704|      2|            2|
|  3390.3755|      3|            3|
|  13547.429|      4|            4|
+-----------+-------+-------------+



## Writing to the Feature Store

### Creating New Feature Groups

In most cases it is recommended that feature groups are created in the UI on Hopsworks and that care is taken in documenting the feature group. 

![Create Feature Group from the UI](./images/create_fg.png "Create Feature Group from the UI")

![Create Feature Group from the UI](./images/create_fg_2.png "Create Feature Group from the UI")

However, sometimes it is practical to create a feature group directly from a spark dataframe and fill in the metadata about the featuregroup later in the UI. This can be done through the `createFeaturegroup()` API function.

Lets create a new featuregroup called **teams_features_spanish** that contains the same contents as the feature group teams_features except the the columns are renamed to spanish

In [17]:
val teamsFeaturesDf = Hops.getFeaturegroup(spark, "teams_features", Hops.getProjectFeaturestore, 1)

teamsFeaturesDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [team_budget: float, team_id: int ... 1 more field]


In [18]:
val teamsFeaturesDf2 = teamsFeaturesDf.withColumnRenamed(
    "team_id", "equipo_id").withColumnRenamed(
    "team_budget", "equipo_presupuesto").withColumnRenamed(
    "team_position", "equipo_posicion")

teamsFeaturesDf2: org.apache.spark.sql.DataFrame = [equipo_presupuesto: float, equipo_id: int ... 1 more field]


In [19]:
teamsFeaturesDf2.show(5)

+------------------+---------+---------------+
|equipo_presupuesto|equipo_id|equipo_posicion|
+------------------+---------+---------------+
|         12957.076|        1|              1|
|         2403.3704|        2|              2|
|         3390.3755|        3|              3|
|         13547.429|        4|              4|
|          9678.333|        5|              5|
+------------------+---------+---------------+
only showing top 5 rows



Lets now create a new featuregroup using the transformed dataframe (we'll explain the statistics part later on in this notebook)

In [20]:
val jobName = null
val dependencies = List[String]().asJava
val primaryKey = null
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null
val description = "a spanish version of teams_features"

Hops.createFeaturegroup(
    spark, teamsFeaturesDf2, "teams_features_spanish", Hops.getProjectFeaturestore,
    1, description, jobName,
    dependencies, primaryKey, descriptiveStats, featureCorr,
      featureHistograms, clusterAnalysis, statColumns, numBins,
      corrMethod, numClusters)

jobName: Null = null
dependencies: java.util.List[String] = []
primaryKey: Null = null
descriptiveStats: Boolean = false
featureCorr: Boolean = false
featureHistograms: Boolean = false
clusterAnalysis: Boolean = false
statColumns: java.util.List[String] = []
numBins: Null = null
corrMethod: Null = null
numClusters: Null = null
description: String = a spanish version of teams_features


By default the new featuregroup will be created in the project's featurestore and the statistics for the new featuregroup will be computed based on the provided spark dataframe. You can configure this behaviour by modifying the default arguments and filling in extra metadata.

The dependencies argument takes a list of HDFS file names that the feature group depends on, i.e when the datasets that a featuregroup depends on have been modified, the feature group should be recomputed. The dependencies can also be updated and viewed in the feature registry UI. 

![Feature group dependencies](./images/deps.png "Feature group dependencies")

![Feature group dependencies](./images/deps2.png "Feature group dependencies")

The jobName argument takes the name of a job that compute the features. If you don't have a job yet, you can add the job later in the feature store UI: 

![Feature group jobs](./images/jobs1.png "Feature group jobs")

![Feature group jobs](./images/jobs2.png "Feature group jobs")

###  Create a New Version of A Feature Group

To create a new version, simply use the `createFeaturegroup` method and change the version argument:

In [21]:
val jobName = null
val dependencies = List[String]().asJava
val primaryKey = null
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null
val description = "a spanish version of teams_features"

Hops.createFeaturegroup(
    spark, teamsFeaturesDf2, "teams_features_spanish", Hops.getProjectFeaturestore,
    2, description, jobName,
    dependencies, primaryKey, descriptiveStats, featureCorr,
      featureHistograms, clusterAnalysis, statColumns, numBins,
      corrMethod, numClusters)

jobName: Null = null
dependencies: java.util.List[String] = []
primaryKey: Null = null
descriptiveStats: Boolean = false
featureCorr: Boolean = false
featureHistograms: Boolean = false
clusterAnalysis: Boolean = false
statColumns: java.util.List[String] = []
numBins: Null = null
corrMethod: Null = null
numClusters: Null = null
description: String = a spanish version of teams_features


You can now see the new version in the feature store UI:

![Create Feature Group Version](./images/create_fg_version.png "Create Feature Group Version")

#### Get the Latest Version of a Feature Group (0 if no version exist)

In [22]:
val latestVersion = Hops.getLatestFeaturegroupVersion("teams_features_spanish", Hops.getProjectFeaturestore)
latestVersion

latestVersion: Int = 2
res21: Int = 2


#### Updating the Metadata Cache

By default the metadata of the feature store will be cached on the client-side after the first query with the feature store, the metadata is used by most queries to the store to figure out how to fetch the features etc. To update the metadata use the following API method:

In [23]:
Hops.updateFeaturestoreMetadataCache(Hops.getProjectFeaturestore)

### Inserting Into Existing Feature Groups

A best practice when working with features in HopsML is to first figure out a model of feature groups and create them  using the Feature Registry UI. This will prepare the feature group schema and create the Hive tables. Once the empty feature groups are created, then you can insert into these tables directly.

Lets first get some sample data to insert

In [24]:
import org.apache.spark.sql._
import spark.implicits._
import org.apache.spark.sql.types._
val sampleData = Seq(
    Row(999, 41251.52f, 1),
    Row(998, 1319.4f, 8),
    Row(997, 21219.1f, 2)
)
val sampleSchema = List(
  StructField("equipo_id", IntegerType, true),
  StructField("equipo_presupuesto", FloatType, true),
  StructField("equipo_posicion", IntegerType, true)
)
val sampleDF = spark.createDataFrame(
  spark.sparkContext.parallelize(sampleData),
  StructType(sampleSchema)
)

import org.apache.spark.sql._
import spark.implicits._
import org.apache.spark.sql.types._
sampleData: Seq[org.apache.spark.sql.Row] = List([999,41251.52,1], [998,1319.4,8], [997,21219.1,2])
sampleSchema: List[org.apache.spark.sql.types.StructField] = List(StructField(equipo_id,IntegerType,true), StructField(equipo_presupuesto,FloatType,true), StructField(equipo_posicion,IntegerType,true))
sampleDF: org.apache.spark.sql.DataFrame = [equipo_id: int, equipo_presupuesto: float ... 1 more field]


In [25]:
sampleDF.show(5)

+---------+------------------+---------------+
|equipo_id|equipo_presupuesto|equipo_posicion|
+---------+------------------+---------------+
|      999|          41251.52|              1|
|      998|            1319.4|              8|
|      997|           21219.1|              2|
+---------+------------------+---------------+



In [26]:
sampleDF.count

res24: Long = 3


Lets inspect the contents of the featuregroup `teams_features_spanish` that we are going to insert the sample data into:

In [27]:
val spanishTeamsFeaturesDf = Hops.getFeaturegroup(spark, "teams_features_spanish", Hops.getProjectFeaturestore, 1)

spanishTeamsFeaturesDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [equipo_presupuesto: float, equipo_id: int ... 1 more field]


In [28]:
spanishTeamsFeaturesDf.show(5)

+------------------+---------+---------------+
|equipo_presupuesto|equipo_id|equipo_posicion|
+------------------+---------+---------------+
|         12957.076|        1|              1|
|         2403.3704|        2|              2|
|         3390.3755|        3|              3|
|         13547.429|        4|              4|
|          9678.333|        5|              5|
+------------------+---------+---------------+
only showing top 5 rows



In [29]:
spanishTeamsFeaturesDf.count

res26: Long = 50


Now we can insert the sample data and verify the new contents of the featuregroup. By default the insert mode is "append", the featurestore is the project's featurestore, the version is 1 and statistics will be updated (we cover statistics later on in this notebook).

In [30]:
val featuregroup = "teams_features_spanish"
val featurestore = Hops.getProjectFeaturestore 
val featuregroupVersion = 1 
val mode = "append"
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null

Hops.insertIntoFeaturegroup(
    spark, 
    sampleDF, 
    featuregroup,
    featurestore,
    featuregroupVersion,
    mode,
    descriptiveStats, 
    featureCorr,
    featureHistograms, 
    clusterAnalysis, 
    statColumns, 
    numBins,
    corrMethod, 
    numClusters
)

featuregroup: String = teams_features_spanish
featurestore: String = demo_featurestore_admin000_featurestore
featuregroupVersion: Int = 1
mode: String = append
descriptiveStats: Boolean = false
featureCorr: Boolean = false
featureHistograms: Boolean = false
clusterAnalysis: Boolean = false
statColumns: java.util.List[String] = []
numBins: Null = null
corrMethod: Null = null
numClusters: Null = null


Lets fetch the updated feature group from the feature store and verify that the update was successful

In [31]:
val spanishTeamsFeaturesUpdatedDf = Hops.getFeaturegroup(spark, "teams_features_spanish", Hops.getProjectFeaturestore, 1)

spanishTeamsFeaturesUpdatedDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [equipo_presupuesto: float, equipo_id: int ... 1 more field]


In [32]:
spanishTeamsFeaturesUpdatedDf.show(5)

+------------------+---------+---------------+
|equipo_presupuesto|equipo_id|equipo_posicion|
+------------------+---------+---------------+
|         12957.076|        1|              1|
|         2403.3704|        2|              2|
|         3390.3755|        3|              3|
|         13547.429|        4|              4|
|          9678.333|        5|              5|
+------------------+---------+---------------+
only showing top 5 rows



In [33]:
spanishTeamsFeaturesUpdatedDf.count

res30: Long = 53


The two supported insert modes are "append" and "overwrite"

In [34]:
val featuregroup = "teams_features_spanish"
val featurestore = Hops.getProjectFeaturestore 
val featuregroupVersion = 1 
val mode = "overwrite"
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null

Hops.insertIntoFeaturegroup(
    spark, 
    sampleDF, 
    featuregroup,
    featurestore,
    featuregroupVersion,
    mode,
    descriptiveStats, 
    featureCorr,
    featureHistograms, 
    clusterAnalysis, 
    statColumns, 
    numBins,
    corrMethod, 
    numClusters
)

featuregroup: String = teams_features_spanish
featurestore: String = demo_featurestore_admin000_featurestore
featuregroupVersion: Int = 1
mode: String = overwrite
descriptiveStats: Boolean = false
featureCorr: Boolean = false
featureHistograms: Boolean = false
clusterAnalysis: Boolean = false
statColumns: java.util.List[String] = []
numBins: Null = null
corrMethod: Null = null
numClusters: Null = null


In [35]:
Hops.getFeaturegroup(spark, "teams_features_spanish", Hops.getProjectFeaturestore, 1).show(5)

+---------+---------------+------------------+
|equipo_id|equipo_posicion|equipo_presupuesto|
+---------+---------------+------------------+
|      998|              8|            1319.4|
|      997|              2|           21219.1|
|      999|              1|          41251.52|
+---------+---------------+------------------+



In [36]:
Hops.getFeaturegroup(spark, "teams_features_spanish", Hops.getProjectFeaturestore, 1).count

res34: Long = 3


## Feature Group Statistics

Statistics about a featuregroup can be useful in the stage of feature engineering and when deciding which features to use for training. If statistics have been computed for a feature group, it can be viewed in the Hopsworks Feature Registry UI. 

This is particularly useful within large organizations where data scientists from different teams can re-use and explore new features by browsing features in the feature store and analyzing the statistics.

![Feature Registry Statistics Visualization](./images/fg_stats_1.png "Feature Registry Statistics Visualization")

![Feature Registry Statistics Visualization](./images/fg_stats_2.png "Feature Registry Statistics Visualization")

![Feature Registry Statistics Visualization](./images/fg_stats_3.png "Feature Registry Statistics Visualization")

![Feature Registry Statistics Visualization](./images/fg_stats_4.png "Feature Registry Statistics Visualization")

![Feature Registry Statistics Visualization](./images/fg_stats_5.png "Feature Registry Statistics Visualization")

As you might have notived earlier in this notebook, both the `insertIntoFeaturegroup` and `createFeaturegroup` methods have arguments for updating the statistics as new data is added. 

You can also use the `updateFeaturegroupStats()` method to update the statistics of a feature group without inserting any new data. By default it will compute all statistics (descriptive, feature correlation, histograms, and cluster analysis), use the project's featurestore, use version 1 of the featuregroup and use all columns for computing statistics:

In [37]:
val featuregroup = "teams_features"
val featurestore = Hops.getProjectFeaturestore
val featuregroupVersion = 1
val descriptiveStats = true
val featureCorr = true
val featureHistograms = true
val clusterAnalysis = true
val statColumns = null // null means all columns will be used
val numBins = 20
val corrMethod = "pearson"
val numClusters = 5
Hops.updateFeaturegroupStats(
    spark, featuregroup, Hops.getProjectFeaturestore, featuregroupVersion,
    descriptiveStats, featureCorr, featureHistograms, clusterAnalysis, statColumns,
    numBins, corrMethod, numClusters
)

featuregroup: String = teams_features
featurestore: String = demo_featurestore_admin000_featurestore
featuregroupVersion: Int = 1
descriptiveStats: Boolean = true
featureCorr: Boolean = true
featureHistograms: Boolean = true
clusterAnalysis: Boolean = true
statColumns: Null = null
numBins: Int = 20
corrMethod: String = pearson
numClusters: Int = 5


If you only want to compute statistics for certain set of columns and exclude surrogate key-columns for example, you can use the argument `statColumns` to specify which columns to include:

In [38]:
val featuregroup = "teams_features"
val featurestore = Hops.getProjectFeaturestore
val featuregroupVersion = 1
val descriptiveStats = true
val featureCorr = true
val featureHistograms = true
val clusterAnalysis = true
val statColumns = List[String]("team_budget", "team_position").asJava
val numBins = 20
val corrMethod = "pearson"
val numClusters = 5
Hops.updateFeaturegroupStats(
    spark, featuregroup, Hops.getProjectFeaturestore, featuregroupVersion,
    descriptiveStats, featureCorr, featureHistograms, clusterAnalysis, statColumns,
    numBins, corrMethod, numClusters
)

featuregroup: String = teams_features
featurestore: String = demo_featurestore_admin000_featurestore
featuregroupVersion: Int = 1
descriptiveStats: Boolean = true
featureCorr: Boolean = true
featureHistograms: Boolean = true
clusterAnalysis: Boolean = true
statColumns: java.util.List[String] = [team_budget, team_position]
numBins: Int = 20
corrMethod: String = pearson
numClusters: Int = 5


## Training Datasets

To group data in the feature store we use three concepts:

- Feature
- Feature group
- Training Dataset

Typically during the feature engineering phase of a machine learning project, you compute a set of features for each type of data that you have, these features are naturally grouped into a documented and versioned **feature group**. 

In practice, it is common that organizations have many different type of datasets that they can extract features from, for example if you are building a recommendation system you might have demographic data about each user as well as user-activity data. 

When you train a machine learning model, you want to use all features that have predictive power and that the model can learn from. At this point, we can create a training dataset of features from several different feature groups and use that for training. That is the purpose of the training dataset abstraction. 

Of course you can always just save a group of features anywhere inside your project, e.g as a csv, or .tfrecords file. However, by using the feature store you can create **managed** training datasets. Managed training datasets will show up in the feature registry UI and will automatically be versioned, documented and reproducible. 

![Feature Engineering Pipeline](./images/pipeline.png "Feature Engineering Pipeline")

Metadata for a training dataset can be created from the Hopsworks UI or directly from the API with the function `createTrainingDataset`. The training datasets in a project are stored in a top-level dataset called `projectName_Training_Datasets`, (i.e `hdfs:///Projects/<ProjectName>/<ProjectName>_Training_Datasets`.

Once a training dataset have been created you can find it in the featurestore UI in hopsworks under the tab `Training datasets`, from there you can also edit the metadata if necessary. 

![Find Training Datasets](./images/find_training_datasets.png "Find Training Datasets")
After a training dataset have been created with the necessary metadata you can save the actual data in the training dataset by using the API function `insertIntoTrainingDataset`.

### Create New Training Dataset

Lets create a dataset called `team_position_prediction` by using a set of relevant features from the featurestore. We will combine features from four different feature groups to form this training dataset: `teams_features`, `attendances_features`, `players_features`, `season_scores_features`.

#### Read Features

In [39]:
val features = List("team_budget", "average_attendance", "average_player_age", "team_position", 
                     "sum_attendance", "average_player_rating", "average_player_worth", "sum_player_age", 
                     "sum_player_rating", "sum_player_worth", "sum_position", "average_position")
val featuresDf = Hops.getFeatures(spark, features, Hops.getProjectFeaturestore)

features: List[String] = List(team_budget, average_attendance, average_player_age, team_position, sum_attendance, average_player_rating, average_player_worth, sum_player_age, sum_player_rating, sum_player_worth, sum_position, average_position)
featuresDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [team_budget: float, average_attendance: float ... 10 more fields]


In [40]:
featuresDf.show(5)

+-----------+------------------+------------------+-------------+--------------+---------------------+--------------------+--------------+-----------------+----------------+------------+----------------+
|team_budget|average_attendance|average_player_age|team_position|sum_attendance|average_player_rating|average_player_worth|sum_player_age|sum_player_rating|sum_player_worth|sum_position|average_position|
+-----------+------------------+------------------+-------------+--------------+---------------------+--------------------+--------------+-----------------+----------------+------------+----------------+
|  16758.066|          3271.934|             25.65|           26|      65438.68|            322.69797|           307.87268|        2565.0|        32269.797|       30787.268|      1103.0|           55.15|
|  3966.3591|         4074.8047|              25.5|           27|      81496.09|            297.79196|           298.78235|        2550.0|        29779.197|       29878.234|      1142.

#### Get the Latest Version of a Training Dataset (0 if no version exist)

In [41]:
val latestVersion = Hops.getLatestTrainingDatasetVersion("team_position_prediction", Hops.getProjectFeaturestore)
latestVersion

latestVersion: Int = 2
res38: Int = 2


#### Save as Training Dataset in TFRecords Format

Now we can create a training dataset from the dataframe with some extended metadata such as schema (automatically inferred). By default when you create a training dataset it will be in "tfrecords" format and statistics will be computed for all features. After the dataset have been created you can view and/or update the metadata about the training dataset from the Hopsworks featurestore UI

In [42]:
val trainingDatasetName = "team_position_prediction"
val jobName = null
val dependencies = List[String]().asJava
val primaryKey = null
val dataFormat = "tfrecords"
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null
val description = "a dataset with features for football teams, used for training a model to predict league-position"
val trainingDatasetVersion = latestVersion + 1
Hops.createTrainingDataset(
    spark, featuresDf, trainingDatasetName, Hops.getProjectFeaturestore,
    trainingDatasetVersion, description, jobName, dataFormat,
    dependencies, descriptiveStats, featureCorr,
      featureHistograms, clusterAnalysis, statColumns, numBins,
      corrMethod, numClusters)

trainingDatasetName: String = team_position_prediction
jobName: Null = null
dependencies: java.util.List[String] = []
primaryKey: Null = null
dataFormat: String = tfrecords
descriptiveStats: Boolean = false
featureCorr: Boolean = false
featureHistograms: Boolean = false
clusterAnalysis: Boolean = false
statColumns: java.util.List[String] = []
numBins: Null = null
corrMethod: Null = null
numClusters: Null = null
description: String = a dataset with features for football teams, used for training a model to predict league-position
trainingDatasetVersion: Int = 3


In [43]:
val latestVersion = Hops.getLatestTrainingDatasetVersion("team_position_prediction_csv", Hops.getProjectFeaturestore)
latestVersion

latestVersion: Int = 2
res40: Int = 2


In [44]:
val trainingDatasetName = "team_position_prediction_csv"
val jobName = null
val dependencies = List[String]().asJava
val primaryKey = null
val dataFormat = "csv"
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null
val description = "a dataset with features for football teams, used for training a model to predict league-position"
val trainingDatasetVersion = latestVersion + 1
Hops.createTrainingDataset(
    spark, featuresDf, trainingDatasetName, Hops.getProjectFeaturestore,
    trainingDatasetVersion, description, jobName, dataFormat,
    dependencies, descriptiveStats, featureCorr,
      featureHistograms, clusterAnalysis, statColumns, numBins,
      corrMethod, numClusters)

trainingDatasetName: String = team_position_prediction_csv
jobName: Null = null
dependencies: java.util.List[String] = []
primaryKey: Null = null
dataFormat: String = csv
descriptiveStats: Boolean = false
featureCorr: Boolean = false
featureHistograms: Boolean = false
clusterAnalysis: Boolean = false
statColumns: java.util.List[String] = []
numBins: Null = null
corrMethod: Null = null
numClusters: Null = null
description: String = a dataset with features for football teams, used for training a model to predict league-position
trainingDatasetVersion: Int = 3


If we now go to the Feature Registy UI in Hopsworks we can see that the training dataset have been created for us and things like versioning, documentation, and recomputation is managed for us. We can also easily edit the metadata from the UI if necesssary.

![Training Dataset UI](./images/training_dataset.png "Training Dataset UI")

###  Create a New Version of A Training Dataset

To create a new version, simply use the `createTrainingDataset` method and specify the version argument:

In [45]:
val trainingDatasetName = "team_position_prediction"
val jobName = null
val dependencies = List[String]().asJava
val primaryKey = null
val dataFormat = "tfrecords"
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null
val description = "a dataset with features for football teams, used for training a model to predict league-position"
val trainingDatasetVersion = Hops.getLatestTrainingDatasetVersion("team_position_prediction", Hops.getProjectFeaturestore) + 1
Hops.createTrainingDataset(
    spark, featuresDf, trainingDatasetName, Hops.getProjectFeaturestore,
    trainingDatasetVersion, description, jobName, dataFormat,
    dependencies, descriptiveStats, featureCorr,
      featureHistograms, clusterAnalysis, statColumns, numBins,
      corrMethod, numClusters)

trainingDatasetName: String = team_position_prediction
jobName: Null = null
dependencies: java.util.List[String] = []
primaryKey: Null = null
dataFormat: String = tfrecords
descriptiveStats: Boolean = false
featureCorr: Boolean = false
featureHistograms: Boolean = false
clusterAnalysis: Boolean = false
statColumns: java.util.List[String] = []
numBins: Null = null
corrMethod: Null = null
numClusters: Null = null
description: String = a dataset with features for football teams, used for training a model to predict league-position
trainingDatasetVersion: Int = 3


You can now see the new version in the feature store UI:

![Create Training Dataset Version](./images/create_td_version.png "Create Training Dataset Version")

### Inserting Into an Existing Training Dataset

Once a dataset have been created, its metadata is browsable in the featurestore registry in the Hopsworks UI. If you don't want to create a new training dataset but just overwrite or insert new data into an existing training dataset, you can use the API function `insertIntoTrainingDataset`.  

**Note**: "append" write mode is not supported for training datasets stored in tfrecords format, only "overwrite"

In [46]:
val trainingDataset = "team_position_prediction_csv"
val featurestore = Hops.getProjectFeaturestore 
val trainingDatasetVersion = latestVersion + 1
val mode = "overwrite"
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null

Hops.insertIntoTrainingDataset(
    spark, 
    featuresDf,
    trainingDataset,
    featurestore,
    trainingDatasetVersion,
    descriptiveStats, 
    featureCorr,
    featureHistograms, 
    clusterAnalysis, 
    statColumns, 
    numBins,
    corrMethod, 
    numClusters,
    mode
)

trainingDataset: String = team_position_prediction_csv
featurestore: String = demo_featurestore_admin000_featurestore
trainingDatasetVersion: Int = 3
mode: String = overwrite
descriptiveStats: Boolean = false
featureCorr: Boolean = false
featureHistograms: Boolean = false
clusterAnalysis: Boolean = false
statColumns: java.util.List[String] = []
numBins: Null = null
corrMethod: Null = null
numClusters: Null = null


### Get Training Dataset Path

After a **managed dataset** have been created, it is easy to share it and re-use it for training various models. For example if the dataset have been materialized in tf-records format you can call the method `getTrainingDatasetPath()` to get the HDFS path and read it directly in your model training (e.g tensorflow) code.

In [47]:
Hops.getTrainingDatasetPath("team_position_prediction_csv", Hops.getProjectFeaturestore, Hops.getLatestTrainingDatasetVersion("team_position_prediction_csv", Hops.getProjectFeaturestore))

res45: String = hdfs://default/Projects/demo_featurestore_admin000/demo_featurestore_admin000_Training_Datasets/team_position_prediction_csv_3/team_position_prediction_csv


### Read Training Dataset into a Spark Dataframe

Typically training datasets are served into deep learning frameworks such as pytorch or tensorflow. However, training datasets can also be read into spark dataframes using the api method `getTrainingDataset()`

In [48]:
Hops.getTrainingDataset(spark, "team_position_prediction_csv", Hops.getProjectFeaturestore, Hops.getLatestTrainingDatasetVersion("team_position_prediction_csv", Hops.getProjectFeaturestore)).show(5)

+-----------+------------------+------------------+-------------+--------------+---------------------+--------------------+--------------+-----------------+----------------+------------+----------------+
|team_budget|average_attendance|average_player_age|team_position|sum_attendance|average_player_rating|average_player_worth|sum_player_age|sum_player_rating|sum_player_worth|sum_position|average_position|
+-----------+------------------+------------------+-------------+--------------+---------------------+--------------------+--------------+-----------------+----------------+------------+----------------+
|  16758.066|          3271.934|             25.65|           26|      65438.68|            322.69797|           307.87268|        2565.0|        32269.797|       30787.268|      1103.0|           55.15|
|  3966.3591|         4074.8047|              25.5|           27|      81496.09|            297.79196|           298.78235|        2550.0|        29779.197|       29878.234|      1142.

### Update Training Dataset Stats

The API is similar to the one for updating the stats of a feature group:

In [49]:
val trainingDataset = "team_position_prediction"
val featurestore = Hops.getProjectFeaturestore
val trainingDatasetVersion = 1
val descriptiveStats = true
val featureCorr = true
val featureHistograms = true
val clusterAnalysis = true
val statColumns = null // null means all columns will be used
val numBins = 20
val corrMethod = "pearson"
val numClusters = 5
Hops.updateTrainingDatasetStats(
    spark, trainingDataset, Hops.getProjectFeaturestore, trainingDatasetVersion,
    descriptiveStats, featureCorr, featureHistograms, clusterAnalysis, statColumns,
    numBins, corrMethod, numClusters
)

trainingDataset: String = team_position_prediction
featurestore: String = demo_featurestore_admin000_featurestore
trainingDatasetVersion: Int = 1
descriptiveStats: Boolean = true
featureCorr: Boolean = true
featureHistograms: Boolean = true
clusterAnalysis: Boolean = true
statColumns: Null = null
numBins: Int = 20
corrMethod: String = pearson
numClusters: Int = 5


### Create a Managed Training Dataset Without Using the API 

To create a **managed** training dataset without using the API, e.g to create a managed training dataset from a .tfrecords file downloaded from Kaggle or from a .csv file from Kaggle, first go to the Feature Registry UI and create a new training dataset and fill in the metadata:

![Create Training Dataset From the UI](./images/create_td_1.png "Create Training Dataset From the UI")

![Create Training Dataset From the UI](./images/create_td_2.png "Create Training Dataset From the UI")

Once the dataset have been created from the UI, you can find that inside the "Training Datasets" folder in your project a new folder for the dataset have showed up that is called `training_datasetname_version`:

![Create Training Dataset From the UI](./images/create_td_3.png "Create Training Dataset From the UI")

Simply upload your dataset inside that folder, e.g you can upload for example a single .csv file or a folder with part-r-X.csv files. **It is important that you name the folder/file the name of your training dataset underscore version, e.g sample_dataset_1 or sample_dataset.csv_1**

## Get Featurestore Metadata
To explore the contents of the featurestore we recommend using the featurestore page in the Hopsworks UI but you can also get the metadata programmatically from the REST API

### Update Metadata Cache

In [50]:
Hops.updateFeaturestoreMetadataCache(Hops.getProjectFeaturestore)

### List all Feature Stores Accessible In the Project

In [51]:
Hops.getProjectFeaturestores

res49: java.util.List[String] = [demo_featurestore_admin000_featurestore]


### List all Feature Groups in a Feature Store

In [52]:
Hops.getFeaturegroups(Hops.getProjectFeaturestore)

res50: java.util.List[String] = [season_scores_features_1, teams_features_spanish_1, teams_features_1, numpy_test_example_1, python_test_example_1, pandas_test_example_1, players_features_1, games_features_1, attendances_features_1, teams_features_spanish_2]


### List all Features in a Feature Store

In [53]:
Hops.getFeaturesList(Hops.getProjectFeaturestore)

res51: java.util.List[String] = [average_position, sum_position, team_id, equipo_id, equipo_posicion, equipo_presupuesto, team_budget, team_id, team_position, col_0, col_1, col_2, col_0, col_1, col_2, average_attendance_test, average_player_age_test, team_budget_test, average_player_age, average_player_rating, average_player_worth, sum_player_age, sum_player_rating, sum_player_worth, team_id, away_team_id, home_team_id, score, average_attendance, sum_attendance, team_id, equipo_id, equipo_posicion, equipo_presupuesto]


### List all Training Datasets in a Feature Store

In [54]:
Hops.getTrainingDatasets(Hops.getProjectFeaturestore)

res52: java.util.List[String] = [team_position_prediction_1, team_position_prediction_csv_1, team_position_prediction_tsv_1, team_position_prediction_parquet_1, team_position_prediction_hdf5_1, team_position_prediction_npy_1, team_position_prediction_2, team_position_prediction_csv_2, team_position_prediction_tsv_2, team_position_prediction_parquet_2, team_position_prediction_hdf5_2, team_position_prediction_npy_2, team_position_prediction_3, team_position_prediction_csv_3]


### Get All Metadata (Features, Feature groups, Training Datasets) for a Feature Store

In [55]:
Hops.getFeaturestoreMetadata(Hops.getProjectFeaturestore)

res53: io.hops.util.featurestore.FeaturegroupsAndTrainingDatasetsDTO = FeaturegroupsAndTrainingDatasetsDTO{featuregroups=[FeaturegroupDTO{, hdfsStorePaths=[hdfs://10.0.2.15:8020/apps/hive/warehouse/demo_featurestore_admin000_featurestore.db/season_scores_features_1]}, FeaturegroupDTO{, hdfsStorePaths=[hdfs://10.0.2.15:8020/apps/hive/warehouse/demo_featurestore_admin000_featurestore.db/teams_features_spanish_1]}, FeaturegroupDTO{, hdfsStorePaths=[hdfs://10.0.2.15:8020/apps/hive/warehouse/demo_featurestore_admin000_featurestore.db/teams_features_1]}, FeaturegroupDTO{, hdfsStorePaths=[hdfs://10.0.2.15:8020/apps/hive/warehouse/demo_featurestore_admin000_featurestore.db/numpy_test_example_1]}, FeaturegroupDTO{, hdfsStorePaths=[hdfs://10.0.2.15:8020/apps/hive/warehouse/demo_featurestore_admin...

## From Raw Data to Features to Training Dataset to Model

Once a training dataset have been materialized, we can use it to train a model. In this section we will train an example model using the training dataset `team_position_prediction` that we just created. We will use the column **"team_position"** as the target to predict.

### Imports

In this example we will use Spark MLLib. However, the feature store is in theory agnostic to which framework or method you use for training the model, it works with PyTorch, Tensorflow, MxNet etc.

In [56]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.regression.LinearRegression

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.regression.LinearRegression


### Constants and HyperParameters

In [48]:
val NUM_ITER = 1000
val ELASTIC_REG_PARAM = 0.8
val REG_LAMBDA_PARAM = 0.3

NUM_ITER: Int = 1000
ELASTIC_REG_PARAM: Double = 0.8
REG_LAMBDA_PARAM: Double = 0.3


## Read TFRecords Dataset into a Spark Dataframe

In [49]:
val dataset_df = Hops.getTrainingDataset(spark, "team_position_prediction", Hops.getProjectFeaturestore, 1)

dataset_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [average_player_rating: float, average_attendance: float ... 10 more fields]


In [50]:
dataset_df.show(5)

+---------------------+------------------+-----------------+------------+----------------+------------------+-------------+--------------------+-----------+----------------+--------------+--------------+
|average_player_rating|average_attendance|sum_player_rating|sum_position|sum_player_worth|average_player_age|team_position|average_player_worth|team_budget|average_position|sum_player_age|sum_attendance|
+---------------------+------------------+-----------------+------------+----------------+------------------+-------------+--------------------+-----------+----------------+--------------+--------------+
|            322.69797|          3271.934|        32269.797|      1103.0|       30787.268|             25.65|           26|           307.87268|  16758.066|           55.15|        2565.0|      65438.68|
|            297.79196|         4074.8047|        29779.197|      1142.0|       29878.234|              25.5|           27|           298.78235|  3966.3591|            57.1|        255

## Convert the Dataframe into Spark MLLib data format

Spark MLLib models typically expect all features to be grouped into a single column instead of having one column per feature, we can use Spark's `VectorAssembler` to group our features together

In [51]:
dataset_df.printSchema

root
 |-- average_player_rating: float (nullable = true)
 |-- average_attendance: float (nullable = true)
 |-- sum_player_rating: float (nullable = true)
 |-- sum_position: float (nullable = true)
 |-- sum_player_worth: float (nullable = true)
 |-- average_player_age: float (nullable = true)
 |-- team_position: long (nullable = true)
 |-- average_player_worth: float (nullable = true)
 |-- team_budget: float (nullable = true)
 |-- average_position: float (nullable = true)
 |-- sum_player_age: float (nullable = true)
 |-- sum_attendance: float (nullable = true)



In [52]:
val transformedDf = new VectorAssembler().
  setInputCols(Array( "average_player_rating","average_attendance", "sum_player_rating", 
                     "sum_position", "sum_player_worth", "average_player_age", "average_player_worth",
                    "team_budget", "average_position", "sum_player_age", "sum_attendance")).
  setOutputCol("features").
  transform(dataset_df).
    drop("average_player_rating").
    drop("average_attendance").
    drop("sum_player_rating").
    drop("sum_player_worth").
    drop("average_player_age").
    drop("average_player_worth").
    drop("team_budget").
    drop("average_position").
    drop("sum_player_age").
    drop("sum_attendance").
    drop("sum_position")

transformedDf: org.apache.spark.sql.DataFrame = [team_position: bigint, features: vector]


In [53]:
transformedDf.printSchema

root
 |-- team_position: long (nullable = true)
 |-- features: vector (nullable = true)



### Define The Model Using Spark MLLib

We will use a linear regression model. In this tutorial we work with so little data that using a larger model does not make sense.

In [54]:
val lr = new LinearRegression().
    setLabelCol("team_position").
    setFeaturesCol("features").
    setMaxIter(NUM_ITER).
    setRegParam(REG_LAMBDA_PARAM).
    setElasticNetParam(ELASTIC_REG_PARAM)

lr: org.apache.spark.ml.regression.LinearRegression = linReg_3e6c52912572


### Train The Model using The Parsed Dataset

In [55]:
val lrModel = lr.fit(transformedDf)

lrModel: org.apache.spark.ml.regression.LinearRegressionModel = linReg_3e6c52912572


### Show Model Training Results

In [56]:
// print the output column and the input column and the truth label
lrModel.transform(transformedDf).select("features", "team_position", "prediction").show()

+--------------------+-------------+------------------+
|            features|team_position|        prediction|
+--------------------+-------------+------------------+
|[322.697967529296...|           26|29.921433049367685|
|[297.791961669921...|           27| 31.64619079360324|
|[881.298278808593...|            9|11.262826067715842|
|[467.793792724609...|           17| 16.82646856958811|
|[1311.23840332031...|            6| 5.466254654943697|
|[502.241394042968...|           16| 18.32368841989546|
|[372.345794677734...|           20|17.566396134916765|
|[226.293045043945...|           40| 37.12318480908224|
|[327.600311279296...|           23|29.786095913268667|
|[205.155975341796...|           39|35.657752460940365|
|[657.393188476562...|           11| 9.628936445599606|
|[231.505447387695...|           33|31.792601813546007|
|[562.398986816406...|           15|10.368459419413842|
|[181.494277954101...|           43|46.636309351594285|
|[297.929351806640...|           28|28.590203155

In [57]:
val trainingSummary = lrModel.summary
println(s"numIterations: ${trainingSummary.totalIterations}")
println(s"objectiveHistory: [${trainingSummary.objectiveHistory.mkString(",")}]")
trainingSummary.residuals.show()
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")
println(s"r2: ${trainingSummary.r2}")

trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@40152c98
numIterations: 56
objectiveHistory: [0.5,0.41124102893970266,0.12518994433439937,0.08777706436856178,0.03913658284735014,0.0390236600487323,0.03902163129558591,0.03901820891749768,0.03899877146854407,0.038996622590291054,0.03898657050161887,0.03898519714099601,0.03898517236997877,0.03898515439811384,0.03898512325511347,0.03898510959220696,0.038985069161690554,0.03898504112882241,0.038985004026080586,0.03898496522452903,0.03898488771326664,0.03898469837570636,0.038983155982796454,0.038983026619741616,0.03898181041529534,0.038981644705107614,0.03898063375147129,0.03898005548853031,0.038978856839679006,0.038977553371813356,0.03897735366410185,0.0389770582901537,0.0389770033693534,0.03897696008493243,0.038976909759638495,0.03897688838840804,0.03897687266655642,0.03897681862522445,0.038976773319025854,0.03897676438201169,0.0389767612367920